<a href="https://colab.research.google.com/github/aminfadaei116/Deep-Learning-Course/blob/master/CA3/Q1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import the Files

In [ ]:
import cv2
from scipy.io import loadmat
import numpy as np
import tqdm
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import linalg as LA
import re
from tqdm.notebook import tqdm
import json
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
# makes it easy to write user-friendly command-line interfaces
import argparse
# contains data structures for multi-dimensional tensors
# and mathematical operations over theme
import torch
# helps us in creating and training of the neural network.
import torch.nn as nn
# contains all the functions in the torch.nn library
import torch.nn.functional as F
# implements various optimization algorithms
import torch.optim as optim
# contains datasets with almost similar API
from torchvision import datasets
# contains common image transformations
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
# provides several methods to adjust the learning rate based on the number of epochs
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score

Load Data
Load Dict

In [ ]:
Test_Data = pd.read_csv('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW3-DataSet/Q1/Data/TestD.csv', header = None, encoding = "ISO-8859-1")
Train_Data = pd.read_csv('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW3-DataSet/Q1/Data/TrainD.csv', header = None, encoding = "ISO-8859-1")[0:5000]
Train_Data.columns = ['label', 'sentence']
Test_Data.columns = ['label', 'sentence']

Here is the Max size of sentence

In [ ]:
sentence_size = []
for i in range(Train_Data.shape[0]):
  sentence_size.append(len(Train_Data.loc[i]['sentence'].split()))
max_size = max(sentence_size)
print(max_size)

32


In [ ]:
with open('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW3-DataSet/Q1/glove.json', 'r') as my_file:
    My_dict = json.load(my_file)

In [ ]:
def string_to_float(str_arr):
  numbers = str_arr.split()
  float_num = [float(num) for num in numbers]
  return float_num

def zerolistmaker(n):
  listofzeros = [0] * n
  return listofzeros

def calc_acc(output1, batch_y, num):
  return ((torch.argmax(output1, dim=1) == batch_y).sum().to(dtype=torch.float32))/num

# this part has been taken from the internet and it is from the https://scikit-learn.org/
def plot_confusion_matrix(y_true, y_pred, 
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
        


def plot_matrix(cm,
                title=None,
                cmap=plt.cm.Blues):
    
    if not title:
        title = 'Confidence matrix'

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
number = np.zeros(300)
for x in My_dict:
  number +=  np.array(string_to_float(My_dict[x]))
avg_arr = number / len(My_dict)

In [ ]:
class Tweet_Data(Dataset):
  def __init__(self, Dict, dataFrame, padding_size):
    self.length = dataFrame.shape[0]
    self.The_dict = Dict
    self.pad_size = padding_size
    self.The_df = dataFrame


  def __getitem__(self, index):
    label = torch.tensor(self.The_df.loc[index]["label"], dtype = torch.uint8)
    tweet = self.The_df.loc[index]["sentence"]
    words = tweet.split()
    temp1 = [(string_to_float(self.The_dict[x]) if x in self.The_dict else avg_arr) for x in words]
    temp2 = [zerolistmaker(300) for _ in range(self.pad_size - len(words))]
    temp3 = torch.FloatTensor(temp1 + temp2)
    return int(label), temp3.view(self.pad_size, 300)
    
  def __len__(self):
    return self.length




In [ ]:
PAD_SIZE = 40 #max_size + 1
DataSize = len(Train_Data)
MyTrainDataSet = Tweet_Data(My_dict, Train_Data[0:int(DataSize*0.95)], PAD_SIZE)
MyValidactionDataSet =Tweet_Data(My_dict, Train_Data[int(DataSize*0.95):].reset_index(), PAD_SIZE)
MyTestDataSet = Tweet_Data(My_dict,Test_Data, PAD_SIZE)
BATCH_SIZE = 16
TrainData = torch.utils.data.DataLoader(dataset=MyTrainDataSet, batch_size=BATCH_SIZE, shuffle=True)
ValidationData = torch.utils.data.DataLoader(dataset=MyValidactionDataSet)
TestData = torch.utils.data.DataLoader(dataset=MyTestDataSet)

In [ ]:
class LSTM_NET(nn.Module):
  def __init__(self):
    super(LSTM_NET, self).__init__()
    self.lstm_layer = nn.LSTM(input_size = 300,  hidden_size = 150, num_layers = 1, batch_first = True)
    self.Fc1 = nn.Linear(150,3)
    self.SM1 = nn.Softmax()
    self.train_loss = []
    self.test_loss = []
    self.val_loss = []
    self.acc_train = []
    self.acc_test = []
    self.acc_val = []

  def forward(self, input2):

    h0 = torch.randn(1, input2.size(0), 150)
    c0 = torch.randn(1, input2.size(0), 150)
    out, (hn,cn) = self.lstm_layer(input2)
    out = self.Fc1(out[:,-1,:])
    out = F.softmax(out, dim=1)
    return  out


def Train_NetWork(model, learning_rate, epoch_number, wr):
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  model.train()
  for epoch in range(epoch_number):
    for i, (labels, inputs) in enumerate(TrainData):
      optimizer.zero_grad()
      output2 = model.forward(inputs)
      loss = F.cross_entropy(output2, labels)
      model.acc_train.append(calc_acc(output2, labels, BATCH_SIZE))
      model.train_loss.append(loss)
      loss.backward()
      optimizer.step()
      print("epoch: ", epoch, " loss for batch: ", loss)
      if i % 200 == 0:
        Test_acc(model)

def Test_Val(model):
  loss = 0
  pred = []
  real = []
  N = len(MyValidactionDataSet)
  print("testing")
  for i in range(N):
    score = MyNetWork.forward(MyValidactionDataSet[i][1].view(-1,40,300))
    label = torch.tensor(MyValidactionDataSet[i][0]).view(-1)
    pred.append(torch.argmax(score).item())
    real.append(label)
    loss += F.cross_entropy(score, label)
  model.val_loss.append(loss)
  print(loss/N)
  plot_confusion_matrix(real, pred)

  loss = 0
  pred = []
  real = []
  N = len(MyTestDataSet)
  for i in range(N):
    score = MyNetWork.forward(MyTestDataSet[i][1].view(-1,40,300))
    label = torch.tensor(MyTestDataSet[i][0]).view(-1)
    pred.append(torch.argmax(score).item())
    real.append(label)
    loss += F.cross_entropy(score, label)
  model.val_loss.append(loss)
  print(loss/N)
  plot_confusion_matrix(real, pred)
 

def Test_acc(model):
  N = len(MyTestDataSet)
  loss = 0
  pred = []
  real = []
  for i in range(N):
    score = model.forward(MyTestDataSet[i][1].view(-1,40,300))
    label = torch.tensor(MyTestDataSet[i][0]).view(-1)
    pred.append(torch.argmax(score).item())
    real.append(label)
    loss += F.cross_entropy(score, label)
  model.acc_test.append(accuracy_score(real, pred))
  model.test_loss.append(loss/N)
  print("The Test loss: ", model.test_loss[-1], "The Test accuracy: ", model.acc_test[-1])
  loss = 0
  pred = []
  real = []
  N = len(MyValidactionDataSet)
  for i in range(N):
    score = model.forward(MyValidactionDataSet[i][1].view(-1,40,300))
    label = torch.tensor(MyValidactionDataSet[i][0]).view(-1)
    pred.append(torch.argmax(score).item())
    real.append(label)
    loss += F.cross_entropy(score, label)
  model.acc_val.append(accuracy_score(real, pred))
  model.val_loss.append(loss/N)

  
def plot_acc(model):
  plt.plot(np.arange(1,len(model.train_loss)+1), model.train_loss)
  plt.ylabel('loss of Train')
  plt.xlabel("number steps")
  plt.grid(True)
  plt.show()
  plt.plot(np.arange(1,len(model.acc_train)+1), model.acc_train)
  plt.ylabel('accuracy of Train')
  plt.xlabel("number steps")
  plt.grid(True)
  plt.show()
  plt.plot(np.arange(1,len(model.val_loss)+1), model.val_loss)
  plt.ylabel('loss of Val')
  plt.xlabel("number steps")
  plt.grid(True)
  plt.show()
  plt.plot(np.arange(1,len(model.acc_val)+1), model.acc_val)
  plt.ylabel('accuracy of Val')
  plt.xlabel("number steps")
  plt.grid(True)
  plt.show()
  plt.plot(np.arange(1,len(model.test_loss)+1), model.test_loss)
  plt.ylabel('loss of Test')
  plt.xlabel("number steps")
  plt.grid(True)
  plt.show()
  plt.plot(np.arange(1,len(model.acc_test)+1), model.acc_test)
  plt.ylabel('accuracy of Test')
  plt.xlabel("number steps")
  plt.grid(True)
  plt.show()

In [ ]:
MyNetWork = LSTM_NET()

In [ ]:
Train_NetWork(MyNetWork, learning_rate=0.001, epoch_number=30,wr= 0.005)

In [ ]:
plot_acc(MyNetWork)

In [ ]:
Test_Val(MyNetWork)

In [ ]:
Test_Val(MyNetWork)